In [1]:
import numpy as np
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris

In [2]:
# From ffnn.ipynb
def countSSError(results, expected_results):
    results = np.array(results)
    expected_results = np.array(expected_results)
    if results.shape != expected_results.shape:
        print("Error: array shape mismatch")
        sys.exit(1)
    else:
        return np.sum(np.square(results - expected_results))

Sklearn Using Iris Dataset

In [3]:
iris = load_iris()

learning_data = np.array(iris.data)
learning_target = np.array(iris.target)

classifier = MLPClassifier(
    hidden_layer_sizes=(5,3,3,2), 
    activation='relu', 
    max_iter=10000, 
    learning_rate='constant', 
    learning_rate_init=0.001, 
    random_state=1
    )

classifier.fit(learning_data, learning_target)
y_pred = classifier.predict(learning_data)

print("SKLearn MLPClassifier with 5,3,3,2 hidden layers and relu activation function")
print(f"Prediction:\n{y_pred}")
print()
print(f"Target:\n{learning_target}")
print()

error = countSSError(y_pred, learning_target)
print(f"Error: {error / len(learning_data) * 100} %")

SKLearn MLPClassifier with 5,3,3,2 hidden layers and relu activation function
Prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 2 1
 1 1 1 2 1 0 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Target:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Error: 4.0 %
